In [1]:
%ll

total 14976
-rw-r--r--  1 kojoidrissa  staff      392 Aug 26 13:51 1_loading.py
-rw-r--r--  1 kojoidrissa  staff      131 Aug 23 16:49 2_get_employee_list_for_loop.py
-rw-r--r--  1 kojoidrissa  staff      159 Aug 23 05:46 2_get_employee_set_comprehension.py
-rw-r--r--  1 kojoidrissa  staff      989 Aug 26 13:51 3_create_employee_dictionary.py
-rw-r--r--  1 kojoidrissa  staff      936 Aug 26 13:51 4_create_output_spreadsheet.py
-rw-r--r--  1 kojoidrissa  staff      387 Aug 26 13:51 5_write_data_to_workbook.py
-rw-r--r--  1 kojoidrissa  staff      162 Aug 22 12:36 6_create_json_object.py
-rw-r--r--  1 kojoidrissa  staff    14238 Aug 24 14:09 PyConAU_2018_spreadsheet.ipynb
-rw-r--r--  1 kojoidrissa  staff     6495 Aug 22 14:21 aggregate_time.json
-rw-r--r--@ 1 kojoidrissa  staff    53440 Aug 24 13:45 build_header.png
-rw-r--r--@ 1 kojoidrissa  staff    95265 Aug 24 13:59 build_output_data.png
-rw-r--r--@ 1 kojoidrissa  staff    56640 Aug 24 13:23 code_1-2.png
-rw-r--r--@ 1 kojoidrissa  st

In [3]:
# %load 1_loading.py
import openpyxl

# get workbook object; the data_only option captures the current value of any formulae
# Your file should be in the same directory as this code, OR you need to provide the FULL path to that file, not just it's name
demo_workbook = openpyxl.load_workbook('demo_workbook.xlsx', data_only= True)

# WB -> WS -> Cell
demo_worksheet = demo_workbook.get_sheet_by_name("clean_data")

In [4]:
type(demo_worksheet)

openpyxl.worksheet.worksheet.Worksheet

In [11]:
# %load 2_get_employee_list_for_loop.py
employee_ids = set()
for row in demo_worksheet.rows: 
    if row[0].value != 'employee_num':
        employee_ids.add(row[0].value)

In [12]:
len(employee_ids)

49

In [13]:
# %load 4_create_employee_dictionary.py
# Create a dictionary of each employee and their info for the month
# This is the slowest step

employee_aggregate = {}
for employee in employee_ids:
    # list comprehension: I'll want to sum the hours later
    hours = [
        row[6].value
        for row in demo_worksheet.rows
        if employee == row[0].value
    ]

    # set comprehension: each employee should belong to only ONE cost center
    cost_center = {row[1].value for row in demo_worksheet.rows if employee == row[0].value}
    division = {str(row[2].value) for row in demo_worksheet.rows if employee == row[0].value}
    manager = {row[3].value for row in demo_worksheet.rows if employee == row[0].value}
    
    assert len(cost_center) == len(division) == len(manager) == 1
    
    employee_aggregate[employee]={
        "hours": sum(hours),
        "cost_center": list(cost_center)[0],
        "division": list(division)[0],
        "manager": list(manager)[0]
    }

# Do I want to demonstrate using dict.update?

In [14]:
len(employee_aggregate)

49

In [16]:
# %load 5_create_output_spreadsheet.py
# Create output workbook, then output worksheet
output_book = openpyxl.Workbook() #OpenPyXL object
output_sheet = output_book.create_sheet("Aggregate Time",0)

# Building the Output Header: Specific Cell References
header = [
    demo_worksheet["A1"].value,
    demo_worksheet["B1"].value,
    demo_worksheet["C1"].value,
    demo_worksheet["D1"].value,
    demo_worksheet["G1"].value
]

# Create output data construct & append header 
output_data = []
output_data.append(header)

# Building new rows, then appending them to output data construct
for employee in employee_aggregate: #iterating over dictionary keys
    new_row = []
    new_row.append(employee)
    new_row.append(employee_aggregate[employee]['cost_center'])
    new_row.append(employee_aggregate[employee]['division'])
    new_row.append(employee_aggregate[employee]['manager'])
    new_row.append(employee_aggregate[employee]['hours'])
    output_data.append(new_row)


In [17]:
type(output_book)

openpyxl.workbook.workbook.Workbook

In [19]:
# %load 6_write_data_to_workbook.py
# Write data to sheet object
for row in output_data:
    rowIn = output_data.index(row)
    for col in range(len(output_data[0])):
        # Adding 1 because spreadsheets count from 1, not 0
        output_sheet.cell(row = rowIn+1, column = col+1).value = output_data[rowIn][col]

# You don't have an actual spreadsheet until you do this
output_book.save(
    filename = "done_pyconau.xlsx"
)

In [22]:
%ll

total 9808
-rw-r--r--  1 kojoidrissa  staff      424 Aug 21 21:14 1_loading.py
-rw-r--r--  1 kojoidrissa  staff      215 Aug 21 19:47 2_get_employee_list_for_loop.py
-rw-r--r--  1 kojoidrissa  staff      135 Aug 21 19:42 3_get_employee_set_comprehension.py
-rw-r--r--  1 kojoidrissa  staff      989 Aug 21 19:36 4_create_employee_dictionary.py
-rw-r--r--  1 kojoidrissa  staff      936 Aug 21 21:19 5_create_output_spreadsheet.py
-rw-r--r--  1 kojoidrissa  staff      387 Aug 21 21:20 6_write_data_to_workbook.py
-rw-r--r--  1 kojoidrissa  staff      162 Aug 21 19:36 7_create_json_object.py
-rw-r--r--  1 kojoidrissa  staff     8745 Aug 21 21:20 Untitled.ipynb
-rw-r--r--  1 kojoidrissa  staff    41786 Aug 21 19:46 demo_workbook.xlsx
-rw-r--r--  1 kojoidrissa  staff     6600 Aug 21 21:20 done_pycaribbean.xlsx
-rw-r--r--  1 kojoidrissa  staff     6600 Aug 21 21:20 done_pyconau.xlsx
-rw-r--r--@ 1 kojoidrissa  staff  1205047 Aug 11 21:54 pyconau_2018_first-timers.key
-rw-r--r--@ 1 kojoidrissa  st

In [24]:
# %load 7_create_json_object.py
# Yes, this import is normally done earlier
import json
with open("aggregate_time.json", 'w') as f:
    json.dump(employee_aggregate, f, sort_keys=True, indent=4)

In [25]:
%ll

total 9824
-rw-r--r--  1 kojoidrissa  staff      424 Aug 21 21:14 1_loading.py
-rw-r--r--  1 kojoidrissa  staff      215 Aug 21 19:47 2_get_employee_list_for_loop.py
-rw-r--r--  1 kojoidrissa  staff      135 Aug 21 19:42 3_get_employee_set_comprehension.py
-rw-r--r--  1 kojoidrissa  staff      989 Aug 21 19:36 4_create_employee_dictionary.py
-rw-r--r--  1 kojoidrissa  staff      936 Aug 21 21:19 5_create_output_spreadsheet.py
-rw-r--r--  1 kojoidrissa  staff      387 Aug 21 21:20 6_write_data_to_workbook.py
-rw-r--r--  1 kojoidrissa  staff      162 Aug 21 19:36 7_create_json_object.py
-rw-r--r--  1 kojoidrissa  staff     8745 Aug 21 21:20 Untitled.ipynb
-rw-r--r--  1 kojoidrissa  staff     6495 Aug 21 21:21 aggregate_time.json
-rw-r--r--  1 kojoidrissa  staff    41786 Aug 21 19:46 demo_workbook.xlsx
-rw-r--r--  1 kojoidrissa  staff     6600 Aug 21 21:20 done_pycaribbean.xlsx
-rw-r--r--  1 kojoidrissa  staff     6600 Aug 21 21:20 done_pyconau.xlsx
-rw-r--r--@ 1 kojoidrissa  staff  12050

In [26]:
%ll

total 9832
-rw-r--r--  1 kojoidrissa  staff      424 Aug 21 21:14 1_loading.py
-rw-r--r--  1 kojoidrissa  staff      215 Aug 21 19:47 2_get_employee_list_for_loop.py
-rw-r--r--  1 kojoidrissa  staff      135 Aug 21 19:42 3_get_employee_set_comprehension.py
-rw-r--r--  1 kojoidrissa  staff      989 Aug 21 19:36 4_create_employee_dictionary.py
-rw-r--r--  1 kojoidrissa  staff      936 Aug 21 21:19 5_create_output_spreadsheet.py
-rw-r--r--  1 kojoidrissa  staff      387 Aug 21 21:20 6_write_data_to_workbook.py
-rw-r--r--  1 kojoidrissa  staff      162 Aug 21 19:36 7_create_json_object.py
-rw-r--r--  1 kojoidrissa  staff    12458 Aug 21 21:22 PyConAU_2018_spreadsheet.ipynb
-rw-r--r--  1 kojoidrissa  staff     6495 Aug 21 21:21 aggregate_time.json
-rw-r--r--  1 kojoidrissa  staff    41786 Aug 21 19:46 demo_workbook.xlsx
-rw-r--r--  1 kojoidrissa  staff     6600 Aug 21 21:20 done_pycaribbean.xlsx
-rw-r--r--  1 kojoidrissa  staff     6600 Aug 21 21:20 done_pyconau.xlsx
-rw-r--r--@ 1 kojoidris